# モジュールのインストールとURLの取得

In [58]:
# flaskのインストール
%pip install flask -q

Note: you may need to restart the kernel to use updated packages.


In [59]:
# ColabのサーバーURLを取得(Colab以外の環境ではローカルホストを返す)

server_port = 5000  # ポート番号

try:
    from google.colab.output import eval_js  # type: ignore
    server_url = eval_js(f"google.colab.kernel.proxyPort({server_port})").strip("/")
except ImportError:
    server_url = f"http://localhost:{server_port}"


# Flaskサーバーの処理を記述

In [60]:
%%writefile flask_app.py
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

app.run()

Overwriting flask_app.py


# サーバーの(再)起動

In [61]:
import time

# IPythonのユーザー名前空間を取得
from IPython import get_ipython  # type: ignore
ipython = get_ipython()
userns = ipython.user_ns

# 以前に起動したサーバーを停止
if "server_process" in userns:
    import signal
    server_process = userns["server_process"]
    if server_process.poll() is None:
        print("Stopping the previous server...")
        try:
            server_process.send_signal(signal.CTRL_C_EVENT)
            try:
                server_process.wait(10)
            except subprocess.TimeoutExpired:
                server_process.kill()
                time.sleep(5)
        except:
            pass


# 前のセルで書き出したflask_app.pyを実行
import subprocess
import sys

# Windowsの場合はpython、それ以外はpython3を使う
if sys.platform == "win32":
    py_name = "python"
else:
    py_name = "python3"

# サーバーを起動
print("Starting the server...")
server_process = subprocess.Popen([py_name, "flask_app.py"], shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

time.sleep(5)

if server_process.poll() is not None:
    print("Failed to start the server.")
    print(server_process.stdout.read().decode())
    print(server_process.stderr.read().decode())
else:
    print("The server is running.")
    print(f"URL: {server_url}")

Stopping the previous server...
Starting the server...
The server is running.
URL: http://localhost:5000


# 実行結果

In [62]:
import IPython

IPython.display.IFrame(src=f"{server_url}/", width="100%", height=500)